In [ ]:
from flask import Flask, render_template, request, redirect, url_for, session
from pymongo import MongoClient
from bson.objectid import ObjectId
from bson.errors import InvalidId

app = Flask(__name__)

# MongoDB 連接（根據你的本地 MongoDB 配置進行調整）
client = MongoClient('mongodb://localhost:27017/')
db = client['local']  # 資料庫名稱
collection = db['startup_log']  # 集合名稱

# 首頁顯示資料列表
@app.route('/')
def index():
    search_query = request.args.get('search', '')
    if search_query:
        search_condition = {
            "$or": [
                {"name": {"$regex": search_query, "$options": "i"}},
                {"description": {"$regex": search_query, "$options": "i"}},
                {"profession": {"$regex": search_query, "$options": "i"}}
            ]
        }
        data = list(collection.find(search_condition))
    else:
        data = list(collection.find({}))
    
    for doc in data:
        doc['_id'] = str(doc['_id'])  # 將 ObjectId 轉為字串
    
    return render_template('index12.html', data=data, search_query=search_query)

# 搜尋功能
@app.route('/search', methods=['POST'])
def search():
    query_text = str(request.form.get("query", ""))
    regex_query = {"$regex": query_text, "$options": "i"}
    query = {"$or": [{field: regex_query} for field in ["name", "description", "hostname", "startTime", "startTimeLocal", "pid"]]}

    results = list(collection.find(query))
    for result in results:
        result['_id'] = str(result['_id'])

    # 搜尋結果，不顯示全部資料
    return render_template('index12.html', data=[], results=results, show_all=session.get('show_all', False))

# 顯示新增資料的表單
@app.route('/create', methods=['GET', 'POST'])
def add_data():
    if request.method == 'POST':
        name = request.form.get('name')
        description = request.form.get('description')
        profession = request.form.get('profession')

        new_doc = {
            'name': name,
            'description': description,
            'profession': profession
        }

        # 動態欄位
        field_names = request.form.getlist('fieldName[]')
        field_values = request.form.getlist('fieldValue[]')
        for i in range(len(field_names)):
            new_doc[field_names[i]] = field_values[i]

        collection.insert_one(new_doc)
        return redirect(url_for('index'))

    return render_template('create11.html')

@app.route('/update/<id>', methods=['GET', 'POST'])
def update(id):
    try:
        # Determine if the ID is an ObjectId or a string
        try:
            object_id = ObjectId(id)
        except InvalidId:
            object_id = id  # Use as a string if not an ObjectId

        if request.method == 'POST':
            # Collect updated data from form
            update_data = {key: value for key, value in request.form.items() if key != '_id'}
            collection.update_one({"_id": object_id}, {"$set": update_data})
            return redirect(url_for('index'))

        # Fetch the document to pre-fill the form
        entry = collection.find_one({"_id": object_id})
        if not entry:
            return "Document not found", 404

        entry['_id'] = str(entry['_id'])  # Ensure _id is a string for template compatibility
        return render_template('update12.html', entry=entry)

    except Exception as e:
        return str(e), 400


@app.route('/delete/<id>', methods=['GET', 'POST'])
def delete(id):
    try:
        # 尝试将传入的 ID 转换为 ObjectId
        try:
            object_id = ObjectId(id)
        except InvalidId:
            object_id = id  # 如果不是 ObjectId 类型，则直接使用字符串 ID

        # 执行删除操作
        result = collection.delete_one({"_id": object_id})

        if result.deleted_count == 0:
            return "Document not found", 404

        return redirect(url_for('index'))  # 删除成功后重定向到首页

    except Exception as e:
        return str(e), 400

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=8080, debug=True, use_reloader=False)



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.0.46:8080
Press CTRL+C to quit
192.168.0.46 - - [16/Nov/2024 20:42:48] "GET / HTTP/1.1" 200 -
192.168.0.46 - - [16/Nov/2024 20:42:55] "GET /update/LAPTOP-02DP50LU-1731156417409 HTTP/1.1" 200 -
192.168.0.46 - - [16/Nov/2024 20:42:58] "GET /update/673160ac2f590b19791b3ce7 HTTP/1.1" 200 -
192.168.0.46 - - [16/Nov/2024 20:43:02] "GET /create HTTP/1.1" 200 -
192.168.0.46 - - [16/Nov/2024 20:43:46] "GET /create HTTP/1.1" 200 -
192.168.0.46 - - [16/Nov/2024 20:43:57] "POST /create HTTP/1.1" 302 -
192.168.0.46 - - [16/Nov/2024 20:43:57] "GET / HTTP/1.1" 200 -
192.168.0.46 - - [16/Nov/2024 20:44:05] "GET /update/6738938daa3f4176591d6f74 HTTP/1.1" 200 -
192.168.0.46 - - [16/Nov/2024 20:44:08] "POST /update/6738938daa3f4176591d6f74 HTTP/1.1" 302 -
192.168.0.46 - - [16/Nov/2024 20:44:08] "GET / HTTP/1.1" 200 -
192.168.0.46 - - [16/Nov/2024 20:44:14] "GET /delete/6738938daa3f4176591d6f74 HTTP/1.1"